In [27]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import sklearn

In [28]:
user_item_interactions_df = pd.read_csv('../tests/test_data/user_item_interactions.csv', parse_dates=['date'])

user_item_interactions_df.head()

,user_id,item_id,date,count
0,U002120,I00006767,2024-07-02,1.0
1,U000748,I00001418,2024-07-02,1.0
2,U004334,I00002579,2024-07-02,1.0
3,U002062,I00000098,2024-07-02,1.0
4,U001214,I00009416,2024-07-02,1.0


In [29]:
# filter interactions to include only users with >= 10 interactions

interactions = user_item_interactions_df.groupby('user_id').filter(lambda x: len(x) >= 10)

# sort for train test split
interactions = interactions.sort_values(['user_id', 'date'], ascending=[True, True]).reset_index(drop=True)

# create ratings column
interactions['rating'] = np.clip(interactions['count'], a_max=3, a_min=1) / 3
interactions.drop(['count', 'date'], axis=1, inplace=True)

unique_users = interactions.user_id.drop_duplicates()
unique_items = interactions.item_id.drop_duplicates()

print(unique_users.shape, unique_items.shape)

interactions.head()

(111,) (2329,)


,user_id,item_id,rating
0,U000066,I00003688,0.333333
1,U000066,I00001166,0.333333
2,U000066,I00000453,0.333333
3,U000066,I00007524,0.333333
4,U000066,I00001241,0.333333


In [30]:
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

interactions['user'] = user_encoder.fit_transform(interactions['user_id'])
interactions['item'] = item_encoder.fit_transform(interactions['item_id'])

interactions.head()

,user_id,item_id,rating,user,item
0,U000066,I00003688,0.333333,0,880
1,U000066,I00001166,0.333333,0,266
2,U000066,I00000453,0.333333,0,102
3,U000066,I00007524,0.333333,0,1860
4,U000066,I00001241,0.333333,0,289
